# Demo of `create_climpred_data`

This demo demonstrates how you setup your raw output from a climate model to match `climpred`'s expectations.

In [4]:
from dask.distributed import Client
import multiprocessing
ncpu = multiprocessing.cpu_count()
threads = 6
nworker = ncpu//threads
print(f'Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}')

Number of CPUs: 48, number of threads: 6, number of workers: 8


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import climpred

In [3]:
from climpred.preprocessing.shared import load_hindcast, set_integer_axis
from climpred.preprocessing.mpi import get_path

Assuming your raw model output is stored in multiple files per member and initialization, `load_hindcast` is a nice wrapper function based on `get_path` designed for the output format of `MPI-ESM` to aggregated all hindcast output into one file as expected by `climpred`.

The basic idea is to look over the output of all members and concatinate, then loop over all initializations and concatinate. Before concatination, it is important to align the `time` dimension.

To reduce the data size, use the `preprocess` function provided to `xr.open_mfdataset` wisely in combination with `set_integer_axis`, e.g. additionally extracting only a certain region or only few variables for a multi-variable input file as in MPI-ESM standard output.

In [48]:
# check the code of load_hindcast
# load_hindcast??

In [49]:
v = "global_primary_production"
def preprocess_1var(ds, v=v):
    """Only leave one variable `v` in dataset """
    return ds[v].to_dataset(name=v).squeeze()

In [8]:
# lead_offset because yearmean output
%time ds = load_hindcast(inits=range(1961, 1965),members=range(1, 3),preprocess=preprocess_1var,get_path=get_path,lead_offset=0)

Processing init 1961 ...
Processing init 1962 ...
Processing init 1963 ...
Processing init 1964 ...
CPU times: user 5.27 s, sys: 2.55 s, total: 7.82 s
Wall time: 9.26 s


In [9]:
# what we need for climpred
ds.coords

Coordinates:
    depth    float64 0.0
    lon      float64 0.0
    lat      float64 0.0
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10
  * member   (member) int64 1 2
  * init     (init) int64 1961 1962 1963 1964

In [13]:
ds[v].data

dask.array<concatenate, shape=(4, 2, 10), dtype=float32, chunksize=(1, 1, 1), chunktype=numpy.ndarray>

In [15]:
# loading the data into memory
# if not rechunk
%time ds = ds.load()

CPU times: user 252 ms, sys: 27 ms, total: 279 ms
Wall time: 231 ms


In [20]:
# you may actually want to use `compute_hindcast` to calculate skill from hindcast.
# for this you also need an `observation` to compare to
# here `compute_perfect_model` compares one member to the ensemble mean of the remain members in turn
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

<xarray.Dataset>
Dimensions:                    (lead: 10)
Coordinates:
    lon                        float64 0.0
    lat                        float64 0.0
    depth                      float64 0.0
  * lead                       (lead) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    global_primary_production  (lead) float64 -0.1276 0.593 ... 0.09196 0.5038
Attributes:
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     4
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    dim:                           ['init', 'member']
    units:                         None
    created:                       2020-02-07 18:11:02

# `intake-esm` for cmorized output

In case you have access to cmorized output of CMIP experiments, consider using `intake-esm`. With the `preprocess` function you can align the `time` dimension of all input files. Finally, `rename_to_climpred_dims` only renames.

In [22]:
from climpred.preprocessing.shared import rename_to_climpred_dims, set_integer_axis

In [24]:
# make to have intake-esm installed
import intake # this is enough for intake-esm to work

In [26]:
# https://github.com/NCAR/intake-esm-datastore/
col_url = "/home/mpim/m300524/intake-esm-datastore/catalogs/mistral-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [27]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'dcpp_init_year',
       'version', 'time_range', 'path'],
      dtype='object')

In [28]:
# load 2 members for 2 inits for one variable from one model
query = dict(experiment_id=[
    'dcppA-hindcast'], table_id='Amon', member_id=['r1i1p1f1', 'r2i1p1f1'], dcpp_init_year=[1970, 1971],
    variable_id='tas', source_id='MPI-ESM1-2-HR')
cat = col.search(**query)
cdf_kwargs = {'chunks': {'time': 12}, 'decode_times': False}

In [29]:
cat.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,r1i1p1f1,Amon,tas,gn,1971.0,v20190906,197111-198112,/work/ik1017/CMIP6/data/CMIP6/DCPP/MPI-M/MPI-E...
1,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,r1i1p1f1,Amon,tas,gn,1970.0,v20190906,197011-198012,/work/ik1017/CMIP6/data/CMIP6/DCPP/MPI-M/MPI-E...
2,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,r2i1p1f1,Amon,tas,gn,1971.0,v20190906,197111-198112,/work/ik1017/CMIP6/data/CMIP6/DCPP/MPI-M/MPI-E...
3,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,r2i1p1f1,Amon,tas,gn,1970.0,v20190906,197011-198012,/work/ik1017/CMIP6/data/CMIP6/DCPP/MPI-M/MPI-E...


In [34]:
def preprocess(ds):
    # extract tiny spatial and temporal subset to make this fast
    ds = ds.isel(lon=[50, 51, 52], lat=[50, 51, 52],
                 time=np.arange(12 * 2))
    # make time dim identical
    ds = set_integer_axis(ds,time_dim='time')
    return ds

In [35]:
dset_dict = cat.to_dataset_dict(
    cdf_kwargs=cdf_kwargs, preprocess=preprocess)

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
             
--> There are 1 group(s)


In [36]:
# get first dict value
_, ds = dset_dict.popitem()
ds.coords

Coordinates:
    height          float64 ...
  * lon             (lon) float64 46.88 47.81 48.75
  * dcpp_init_year  (dcpp_init_year) float64 1.97e+03 1.971e+03
  * time            (time) int64 1 2 3 4 5 6 7 8 9 ... 17 18 19 20 21 22 23 24
  * lat             (lat) float64 -42.55 -41.61 -40.68
  * member_id       (member_id) <U8 'r1i1p1f1' 'r2i1p1f1'

In [37]:
# rename to comply with climpred's required dimension names
ds = rename_to_climpred_dims(ds)

In [37]:
# what we need for climpred
ds.coords

Coordinates:
    height   float64 ...
  * lon      (lon) float64 46.88 47.81 48.75
  * init     (init) float64 1.97e+03 1.971e+03
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 15 16 17 18 19 20 21 22 23 24
  * lat      (lat) float64 -42.55 -41.61 -40.68
  * member   (member) <U8 'r1i1p1f1' 'r2i1p1f1'

In [39]:
ds['tas'].data

dask.array<concatenate, shape=(2, 2, 24, 3, 3), dtype=float32, chunksize=(1, 1, 12, 3, 3), chunktype=numpy.ndarray>

In [44]:
# loading the data into memory
# if not rechunk
# this is here quite fast before we only select 9 grid cells
%time ds = ds.load()

CPU times: user 201 ms, sys: 63 ms, total: 264 ms
Wall time: 203 ms


In [45]:
# you may actually want to use `compute_hindcast` to calculate skill from hindcast.
# for this you also need an `observation` to compare to
# here `compute_perfect_model` compares one member to the ensemble mean of the remain members in turn
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

/work/mh0727/m300524/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/xskillscore/core/np_deterministic.py:182: RuntimeWarning: invalid value encountered in true_divide
  r = r_num / r_den


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 3, lead: 24, lon: 3)
Coordinates:
    height     float64 2.0
  * lon        (lon) float64 46.88 47.81 48.75
  * lead       (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 16 17 18 19 20 21 22 23 24
  * lat        (lat) float64 -42.55 -41.61 -40.68
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lead, lat, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    lon_bnds   (lead, lon, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    time_bnds  (lead, bnds) float64 nan nan nan nan nan ... nan nan nan nan nan
    tas        (lead, lat, lon) float32 0.933643 0.88438606 ... -0.8581106
Attributes:
    variable_id:                   tas
    external_variables:            areacella
    Conventions:                   CF-1.7 CMIP-6.2
    cmor_version:                  3.5.0
    nominal_resolution:            100 km
    references:                    MPI-ESM: Mauritsen, T. et al. (2019), Deve...
    mip_era:                       CMIP6
    project_id:                    CMIP6
    physics_index:                 1
    branch_method:                 standard lagged initialization
    intake_esm_varname:            tas
    source_id:                     MPI-ESM1-2-HR
    source:                        MPI-ESM1.2-HR (2017): \naerosol: none, pre...
    grid:                          spectral T127; 384 x 192 longitude/latitude
    experiment_id:                 dcppA-hindcast
    grid_label:                    gn
    title:                         MPI-ESM1-2-HR output prepared for CMIP6
    source_type:                   AOGCM
    history:                       2019-09-06T14:20:04Z ; CMOR rewrote data t...
    contact:                       cmip6-mpi-esm@dkrz.de
    activity_id:                   DCPP
    forcing_index:                 1
    tracking_id:                   hdl:21.14100/f41d2fe5-bb1c-49d0-8afa-0cb9e...
    initialization_index:          1
    realm:                         atmos
    frequency:                     mon
    institution_id:                MPI-M
    data_specs_version:            01.00.30
    product:                       model-output
    license:                       CMIP6 model data produced by MPI-M is lice...
    table_id:                      Amon
    table_info:                    Creation Date:(09 May 2019) MD5:e6ef8ececc...
    institution:                   Max Planck Institute for Meteorology, Hamb...
    experiment:                    hindcast initialized based on observations...
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     2
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    dim:                           ['init', 'member']
    units:                         None
    created:                       2020-02-07 18:16:21